<h1 style="font-family:Times New Roman">Матрица расстояний</h1>

*Nikolay Sitala 2021, nsitala@ukr.net* 
>Геокодер Nominatim
>http://map.project-osrm.org

[Как преобразовать файлы Excel в формат CSV](https://office-guru.ru/excel/kak-preobrazovat-excel-v-csv-i-eksportirovat-faily-excel-v-format-csv-utf-8-456.html#2>)

<h3 style="font-family: 'Times New Roman'">Скачайте пример файла CSV с координатами</h3>

In [149]:
import io
import pandas as pd
import base64
from IPython.display import FileLink
from ipywidgets import HTML

def create_download_link(df, title = "Скачать пример в CSV file ", filename = "data.csv"):  
    csv = df.to_csv(sep=';', encoding='utf-8-sig', index=False)
    b64 = base64.b64encode(csv.encode('utf-8-sig'))
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

csv_data = pd.read_csv("csv_coords.test", delimiter=';', encoding ='utf-8')
create_download_link(csv_data)

HTML(value='<a download="data.csv" href="data:text/csv;base64,77u/bG9jYXRpb247bG9udGl0dWRlO2xhdGl0dWRlDQrQltC1…

<h2 style="font-family: 'Times New Roman'">Загрузите файл формат CSV c координатами</h2>

In [21]:
import ipywidgets as widgets

uploader = widgets.FileUpload(
    accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
uploader

FileUpload(value={}, accept='.csv', description='Upload')

In [153]:
import io
import openpyxl
from pandas import ExcelFile
import pandas as pd

csv_data = pd.read_csv(io.BytesIO(uploader.data[-1]), delimiter=';', encoding ='utf-8')
csv_data

,location,lontitude,latitude
0,"Жерноклевы, ЧЕРКАССКАЯ",31.935005,49.969062
1,"Зарубинцы, ЧЕРКАССКАЯ",29.955307,49.126683
2,"Звенигородка, ЧЕРКАССКАЯ",30.968338,49.070364
3,"Здобуток Жовтня, ЧЕРКАССКАЯ",30.749146,48.868504
4,"Золотоноша, ЧЕРКАССКАЯ",32.042598,49.669020
5,"Ирдынь, ЧЕРКАССКАЯ",31.677246,49.367147
6,"Ирклиев, ЧЕРКАССКАЯ",32.338096,49.518256
7,"Кайтановка, ЧЕРКАССКАЯ",31.082583,48.891665
8,"Канев, ЧЕРКАССКАЯ",31.459825,49.749587


<h2 style="font-family: 'Times New Roman'">Cкрипт создания матрицы расстояний</h2>

In [154]:
## http://map.project-osrm.org/

import os
import time
import json
import pprint
import requests

from openpyxl import Workbook, load_workbook
from openpyxl.worksheet import worksheet
from tqdm.notebook import tqdm, trange

wb = Workbook()


def get_distance_osrm(coordinates):

    # coordinates = '30.613808,46.471085;29.234447,47.997272'

    r1 = f'http://router.project-osrm.org/route/v1/driving/{coordinates}?overview=false'

    i = 8
    while i > 0:
        res = requests.get(r1, timeout=None)
        x = res.headers.get('Cache-Control')
        if x == 'max-age=0':
            break
        else:
            break
            i -= 1
            time.sleep(10)

    res1 = res.content
    res2 = res1.decode(encoding='UTF-8', errors='strict')
    res_json = json.loads(res2)

    return res_json


def import_data_csv():
    
    global csv_data
    
    locate = list(csv_data['location'])
    lat = list(csv_data['latitude'])
    lon = list(csv_data['lontitude'])
    
    for b in zip(locate, lat, lon):
        yield(b)


def get_progress(mtx):
    
    with trange(mtx) as pbar:
        pbar.container.children[-2].style.bar_color = '#003366'  # or 'yellow'
        for i in pbar:
            yield 0
        
        
def get_data_matrix():

    j = 0
    coords_pull = []
    matrix_out = []
    
    for z in import_data_csv():
        j += 1
        coords_pull.append([j, z[0], z[1], z[2]])

    while coords_pull:
        s = coords_pull.pop(0)
        matrix_out.extend([[s, b] for b in [h for h in coords_pull]])

    return matrix_out


def add_data_list(matrix_out):
    
    wb.create_sheet("array")
    wsarray = wb['array']
    
    br = get_progress(len(matrix_out)*2)
    
    for b in matrix_out:
        
        r = [str(b[0][3]), str(b[0][2]), str(b[1][3]), str(b[1][2])]
        coordinates = f'{r[0]},{r[1]};{r[2]},{r[3]}'
        res = get_distance_osrm(coordinates)
        if res:
            next(br)
            y = res['routes'][0]
            legs = y['legs'][0]
            dist_meters = legs['distance']
            dist_km = round(dist_meters / 1000, 3)

            wsarray.cell(row=b[0][0], column=b[1][0]).value = dist_km
#             print(b[0][0], b[0][1], b[1][0], b[1][1], dist_km)

        r = [str(b[1][3]), str(b[1][2]), str(b[0][3]), str(b[0][2])]
        coordinates = f'{r[0]},{r[1]};{r[2]},{r[3]}'
        res = get_distance_osrm(coordinates)
        if res:
            next(br)
            y = res['routes'][0]
            legs = y['legs'][0]
            dist_meters = legs['distance']
            dist_km = round(dist_meters / 1000, 3)

            wsarray.cell(row=b[1][0], column=b[0][0]).value = dist_km
#             print(b[1][0], b[1][1], b[0][0], b[0][1],  dist_km)

    try:
        next(br)
    except Exception:
        pass
                   
    wsarray.insert_cols(0)
    
    for i, b in enumerate(list(csv_data['location'])):
        wsarray.cell(row=i+1, column=1).value = b

    return wsarray


if __name__ == '__main__':

    matrix_out = get_data_matrix()
    sheet_openpyxl = add_data_list(matrix_out)
    head_name = list(csv_data['location'])
    head_name.insert(0, '')
    new_df = pd.DataFrame(sheet_openpyxl.values, columns=head_name)

  0%|          | 0/72 [00:00<?, ?it/s]

<h2 style="font-family:Times New Roman">Скачать матрицу в CSV file</h2>

In [155]:
import base64
from IPython.display import FileLink
from ipywidgets import HTML

def create_download_link(df, title = "Скачать матрицу в CSV file ", filename = "data.csv"):  
    csv = df.to_csv(sep=';', encoding='utf-8-sig', index=False)
    b64 = base64.b64encode(csv.encode('utf-8-sig'))
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(new_df)

HTML(value='<a download="data.csv" href="data:text/csv;base64,77u/O9CW0LXRgNC90L7QutC70LXQstGLLCDQp9CV0KDQmtCQ…